# 설정

In [5]:
# 파이썬 ≥ 3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥ 0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

SVM처럼 <b>결정 트리</b><sup>decision tree</sup>는 분류와 회귀 작업 그리고 다중출력 작업도 가능한 다재다능한 머신러닝 알고리즘이다. 또한 데이터의 스케일링이나 정규화 등의 사전 가공의 영향이 매우 적다. 하지만 예측 성능을 향상시키기 위해 복잡한 규칙 구조를 가져야 하며, 이로 인한 과대적합이 발생해 반대로 예측 성능이 저하될 수도 있다. 하지만 이러한 단점이 앙상블 기법에서는 오히려 장점으로 작용한다.

결정 트리는 또한 설명이 중요할 때 아주 유용한 모델이다. 결정 트리라는 이름처럼 일련의 질문에 대한 결정을 통해 데이터를 분해하는 모델로 생각할 수 있다. 이와 같은 이유로 학습한 모델을 시각화하여 사람이 해석할 수 있다는 점이 가장 큰 특성이다.

다음은 앞에서 언급한 것 외에 추가적인 특성이다.

<ul>
    <li>범주형 변수나 데이터의 누락값(계측 오류 등으로 값이 존재하지 않는 경우)이 있어도 용인된다.</li>
    <li>비선형 문제에는 적용할 수 있지만, 선형 분리 문제는 잘 풀지 못한다.</li>
    <li>데이터 분포가 특정 클래스에 쏠려 있으면 잘 풀지 못한다.</li>
    <li>데이터의 작은 변화에도 결과가 크게 바뀌기 쉽다.</li>
    <li>예측 성능은 보통이다.</li>
    <li>배치 학습으로만 학습할 수 있다.</li>
 </ul>

이 장에서는 결정 트리의 훈련, 시각화, 예측 방법에 대해 먼저 살펴보겠다. 그런 다음 사이킷런의 CART 훈련 알고리즘을 둘러보고 트리에 규제를 가하는 방법과 회귀 문제에 적용하는 방법을 배우도록 하겠다. 마지막으로 결정 트리의 제약 사항에 관해 이야기하겠다.

# 개요

결정 트리는 훈련 데이터에 있는 특성을 기반으로 샘플의 클래스 레이블을 추정할 수 있는 일련의 질문을 학습한다. 따라서 데이터의 어떤 기준을 바탕으로 질문을 만들어야 가장 효율적인 분류가 될 것인가가 알고리즘의 성능을 크게 좌우한다. 이 질문은 범주형 변수는 물론 실수형 특성에도 적용된다. 예를 들어 꽃받침 너비 특성 축에 기준 값을 정하고 "꽃받침 너비가 2.8cm보다 큰가?"라는 예/아니요 질문을 할 수 있다.

결정 알고리즘을 사용하면 트리의 루트<sup>root</sup>에서 시작해서 <b>정보 이득</b><sup>Information Gatin, IG</sup>이 최대가 되는 특성으로 데이터를 나눈다. 정보 이득에 대해서는 후에 자세히 다룰 예정이다. 반복 과정을 통해 리프 노드<sup>leaf node</sup>가 순수해질 때까지 모든 자식 노드에서 이 분할 작업을 반복한다. 즉, 이 노드의 모든 샘플은 동일한 클래스에 속한다. 하지만 실제로 이렇게 하면 노드가 많은 깊은 트리가 만들어지고 과대적합될 가능성이 높다. 일반적으로 트리의 최대 깊이를 제한하여 트리를 <b>가지치기</b><sup>pruning</sup>한다.